In [ ]:
import pickle
from pathlib import Path

import os
# 작업 디렉토리를 crop_pkl로 변경
ROOT = r"D:\golfDataset\dataset\crop_pkl"
os.chdir(ROOT)

def load_all_annotations(pkl_path):
    with open(pkl_path, 'rb') as f:
        data = pickle.load(f)
    return data['annotations']

def make_mmaction_pkl(train_pkl_list, valid_pkl_list, test_pkl_list, out_train, out_test):
    # 1. train/valid
    train_annos, train_dirs, valid_dirs = [], [], []
    for pkl in train_pkl_list:
        annos = load_all_annotations(pkl)
        train_annos.extend(annos)
        train_dirs.extend([anno['frame_dir'] for anno in annos])
    for pkl in valid_pkl_list:
        annos = load_all_annotations(pkl)
        train_annos.extend(annos)
        valid_dirs.extend([anno['frame_dir'] for anno in annos])
    split = {
        'xsub_train': train_dirs,
        'xsub_val': valid_dirs
    }
    train_data = {
        'split': split,
        'annotations': train_annos
    }
    with open(out_train, 'wb') as f:
        pickle.dump(train_data, f)
    print(f"[INFO] Wrote train.pkl ({len(train_annos)} samples)")

    # 2. test
    test_annos, test_dirs = [], []
    for pkl in test_pkl_list:
        annos = load_all_annotations(pkl)
        test_annos.extend(annos)
        test_dirs.extend([anno['frame_dir'] for anno in annos])
    split = {
        'xsub_val': test_dirs
    }
    test_data = {
        'split': split,
        'annotations': test_annos
    }
    with open(out_test, 'wb') as f:
        pickle.dump(test_data, f)
    print(f"[INFO] Wrote test.pkl ({len(test_annos)} samples)")

# 사용 예시
if __name__ == '__main__':
    # 실제 파일명에 맞게 경로를 지정하세요
    train_pkl_list = ['skeleton_dataset_train.pkl']
    valid_pkl_list = ['skeleton_dataset_valid.pkl']
    test_pkl_list  = ['skeleton_dataset_test.pkl']

    make_mmaction_pkl(
        train_pkl_list, valid_pkl_list, test_pkl_list,
        out_train='train.pkl',
        out_test='test.pkl'
    )

[INFO] Wrote train.pkl (1621 samples)
[INFO] Wrote test.pkl (289 samples)


In [3]:
import pickle
import numpy as np
from pathlib import Path

# 파일 경로 지정
PKL_PATHS = [
    ("D:\\golfDataset\\dataset\\crop_pkl\\train.pkl", "D:\\golfDataset\\dataset\\crop_pkl\\train_unnorm.pkl"),
    ("D:\\golfDataset\\dataset\\crop_pkl\\test.pkl",  "D:\\golfDataset\\dataset\\crop_pkl\\test_unnorm.pkl"),
]

def unnormalize_keypoints(annotations, target_mean, target_std):
    # annotations: list of dicts, 각 dict에 'keypoint' (T, V, 2)
    for ann in annotations:
        kp = np.array(ann['keypoint'])  # (T, V, 2)
        cur_mean = np.mean(kp)
        cur_std = np.std(kp)
        kp = (kp - cur_mean) / (cur_std + 1e-8)
        kp = kp * target_std + target_mean
        ann['keypoint'] = kp
    return annotations

# ntu60_2d.pkl의 분포에 맞추기
target_mean = np.array([367.29764, 188.13676])
target_std = np.array([441.61224, 314.72678])

for pkl_in, pkl_out in PKL_PATHS:
    with open(pkl_in, 'rb') as f:
        data = pickle.load(f)
    data['annotations'] = unnormalize_keypoints(data['annotations'], target_mean, target_std)
    with open(pkl_out, 'wb') as f:
        pickle.dump(data, f)
    print(f"[INFO] Saved: {pkl_out}")

[INFO] Saved: D:\golfDataset\dataset\crop_pkl\train_unnorm.pkl
[INFO] Saved: D:\golfDataset\dataset\crop_pkl\test_unnorm.pkl
